In [20]:
# Podstawowe biblioteki do wykonania zadania
from random import uniform,choice
from matplotlib import pyplot as plt
from time import time
import numpy as np
from json import dumps

Metoda_1._Generowanie_10^5_losowych_punktów_na_przedziale_<-10^3,10^3>_

In [3]:
def First_generator(n):
    return [ (uniform(-10**3,10**3),uniform(-10**3,10**3)) for _ in range(n) ]

In [ ]:
N = 10**5
points1 = First_generator(N)
# Zmniejszona wartość wielkości, dla lepszej czytelności punktów (widać, że jes to jednolity rozkład, nie posiada widocznych skupisk)
plt.figure(figsize=(5,5))
plt.scatter(*zip(*points1),s=[0.1 for _ in range(N)])
plt.show()

Medota 2. Generowanie 10^5 losowych punktów na przedziale <-10^14,10^14> 

In [5]:
def Second_generator(n):
    return [ (uniform(-10**14,10**14),uniform(-10**14,10**14)) for _ in range(n)  ]

In [ ]:
N = 10**5
points2 = Second_generator(N)
# Zmniejszona wartość wielkości, dla lepszej czytelności punktów (widać, że jes to jednolity rozkład, nie posiada widocznych skupisk)
plt.figure(figsize=(5,5))
plt.scatter(*zip(*points2),s=[0.01 for _ in range(N)])
plt.show()

Medota 3. Generowanie 10^5 losowych punktów na przedziale <-10^14,10^14> 

In [7]:
def Third_generator(n):
    result = [ ]
    for _ in range(n):
        # x = uniform(-100,100)
        # result.append((x,uniform(-sqrt(100**2 - x**2),sqrt(100*2 - x**2)))
        theta = uniform(0,2*np.pi)
        R = 100
        result.append((R*np.cos(theta),R*np.sin(theta)))
    return result

In [ ]:
N = 10**3
points3 = Third_generator(N)
plt.figure(figsize=(5,5))
plt.scatter(*zip(*points3),s=[2 for _ in range(N)])
plt.show()

Medota 4. Generowanie 10^5 losowych punktów na przedziale <-10^14,10^14> 

In [9]:
def Fourth_generator(n):
    result = []
    for _ in range(n):
        #t = uniform(-500,500)
        #result.append((t*2,t*0.1))
        x = uniform(-1000,1000)
        result.append((x,0.05*x+0.05))
    return result

In [ ]:
N = 10**3
points4 = Fourth_generator(N)
plt.figure(figsize=(5,5))
plt.scatter(*zip(*points4),s=[4 for _ in range(N)])
plt.show()

Własny wyznacznik 1.

In [11]:
def det3x3(a,b,c):
    return (b[0]*c[1]-c[0]*b[1])-(a[0]*c[1]-a[1]*c[0])+(a[0]*b[1]-b[0]*a[1])

In [12]:
def det2x2(a,b,c):
    return ((a[0]-c[0])*(b[1]-c[1]))-((b[0]-c[0])*(a[1]-c[1]))

In [13]:
def make_matrix_3x3(a,b,c):
    return np.matrix([[a[0],a[1],1],[b[0],b[1],1],[c[0],c[1],1]])
def det_np_3x3(a,b,c):
    return np.linalg.det(make_matrix_3x3(a,b,c))

In [14]:
def make_matrix_2x2(a,b,c):
    return np.matrix([[a[0]-c[0],a[1]-c[1]],[b[0]-c[0],b[1]-c[1]]])
def det_np_2x2(a,b,c):
    return np.linalg.det(make_matrix_2x2(a,b,c))

In [18]:
# ======================================= IMPORTANT =======================================

WRITE_TO_FILES = False

#  ================ Change this to False to avoid cluttering the directory ================

Poniżej znajduje się "serce" programu, funkcja calculate liczy pozycje punktów, nadaje numery kolorów, korzysta z dekodowania kolorów i rysuje je na wykresie.
Funkcje run_all_preprocessed i run_all_functions generują wyniki dla każdego przypadku

In [ ]:
SIZES = [4,1,0.1]

def decode_colors(orient):
    cols = [ (0,0,1), (0,1,0) , (1,0,0) ]
    res = []
    for i in range(len(orient)):
        res.append(cols[orient[i]])
    return res

def calculate(points=[],n=100,epsilon=0,a=(-1.0,0.0),b=(1.0,0.1),detFun=det3x3,dot_size=1,show_line = True,save_to_image=False,name_params=("Name","float32","det1")):
    states = [0,0,0]
    generator_name,type_name,det_name = name_params
    orient = [ None for _ in range(n) ]
    for i in range(n):
        orient[i] = orientation(points[i],a,b,epsilon,detFun)
        states[orient[i]] += 1
    z = zip(*points)
    decoded = decode_colors(orient)
    plt.figure(figsize=(5,5))
    plt.scatter(*z,c=decoded,s=[SIZES[dot_size] for _ in range(n)])
    if show_line: 
        plt.axline(a, b, markersize=2)
    if save_to_image:
        name = generator_name.replace('$', '').replace('\\','')
        det = det_name.replace('\\', '')
        plt.savefig(f"{name}_{str(epsilon)}_{type_name}_{det}.png")
    plt.show()
    return states

def orientation(a,b,c,epsilon,detFun):
    det = detFun(a,b,c)
    if 10**-8  < det < 10**-4:
        print(f"Determinant value: {det}")
    if det <= epsilon and det >= -epsilon:
        return 0 # Colinear
    elif det > epsilon:
        return 1 # Positive orientation to the line
    else:
        return 2 # Negative orientation to the line


def run_all_preprocessed(points_list,epsilons,dets,types,save_to_image=False):
    for points,n,generator_name,size,show_line in points_list:
        # with open(f"{generator_name}.txt","w") as f:
        #     f.write("epsilonm,typ numeryczy,wyznacznik,czas,poniżej prostej,powyżej prostej,na prostej\n")
        results = []
        numbers_to_dets = { 
            "1.": "Własna implementacja $3 \\times 3$",
            "2.": "Własna implementacja $2 \\times 2$", 
            "3.": "Wyznacznik macierzy $2 \\times 2$ z numpy",  
            "4.": "Wyznacznik macierzy $3 \\times 3$ z numpy"  
        }
            
        name = generator_name.replace('$', '').replace('\\','')
        tex_filename = f"{name}.tex"
        plt.figure(figsize=(5,5))
        plt.scatter(*zip(*points),s=[SIZES[size] for _ in range(n)])
        plt.savefig(name + ".png")

        with open(tex_filename, "w") as f:
            f.write("\\begin{table}[H]\n")
            f.write("    \\centering\n")
            f.write("    \\small\n")
            f.write(f"    \\caption{{Tabela różnic czasów obliczeń i tolerancji dla generatora {generator_name.replace('_',' ')}}}\n")
            f.write("    \\resizebox{\\textwidth}{!}{\n")
            f.write("    \\begin{tabular}{@{}llccccc@{}}\n")
            f.write("        \\toprule\n")
            f.write("        Tolerancja \\epsilon & Typ numeryczny & Numer wyznacznika & Czas obliczeń (s) & Punkty powyżej prostej & Punkty poniżej prostej & Punkty na prostej \\\\ \\midrule\n")
        for type_, type_name in types:
            points_typed = type_(points) 
            for epsilon in epsilons:
                for detFun,det_num in dets:
                    # Thick tolerance, tests if the line is properly set and if the code is working
                    # epsilon = 10
                    # detFun = det2
                    start = time()
                    states = calculate(points=points_typed,n=n,epsilon=epsilon,detFun=detFun,dot_size=size,show_line=show_line, name_params=(generator_name,type_name,numbers_to_dets[det_num]) ,save_to_image=save_to_image)
                    end = time()
                    elapsed_time = end - start
                    print (f"For {generator_name} elapsed time {elapsed_time} for epsilon {epsilon} using type {type_name} and determinant {numbers_to_dets[det_num]}\nAbove: {states[1]} Below: {states[2]} Co-linear: {states[0]} ")
                    # with open(f"{generator_name}.txt","a") as f:
                    #     f.write(f"{epsilon},{type_name},{det_name},{end-start},{states[1]},{states[2]},{states[0]}\n")
                    results.append({
                        "generator_name": generator_name,
                        "epsilon": epsilon,
                        "type_name": type_name,
                        "det_name": det_num,
                        "elapsed_time": elapsed_time,
                        "above_line": states[1],
                        "below_line": states[2],
                        "collinear": states[0]
                    })
        with open(tex_filename,"a") as f:
            for result in results:
                f.write(f"        {result['epsilon']} & {result['type_name']} & {result['det_name']} & {result['elapsed_time']:.4f} & {result['above_line']} & {result['below_line']} & {result['collinear']} \\\\\n")
            
            f.write("        \\bottomrule\n")
            f.write("    \\end{tabular}\n}\n")
            f.write("    \\small\n")
            f.write("    \\caption*{")
            legend = dumps(numbers_to_dets).replace('{','').replace('}','',).replace(',','\\\\').replace(':', ' -')
            f.write(f"    {legend}")
            f.write("}\n    \\normalsize\n")
            f.write("\\end{table}\n")

def run_all_functions(generator_list,epsilons,dets,types):
    points_set = []
    for generator_function,n,generator_name,size,show_line in generator_list:
        # with open(f"{generator_name}.txt","w") as f:
        #     f.write("epsilon,det,time,s1,s2,s3\n")
        points_set.append((generator_function(n),n,generator_name,size,show_line))        
    run_all_preprocessed(points_set,epsilons,dets,types)

# Invariant 
epsilons = [10**-14,10**-8] # , 10**-12, 10**-10]
types = [(np.float32,"float32"), (np.float64,"float64")]

# # Polish versions of determinants
# dets_pl = [
#     (det3x3, "Własna implementacja $3 \\times 3$"),
#     (det2x2, "Własna implementacja $2 \\times 2$"),
#     (det_np_2x2, "Wyznacznik macierzy $2 \\times 2$ z numpy"),
#     (det_np_3x3, "Wyznacznik macierzy $3 \\times 3$ z numpy")
# ]

# # English versions of determinants
# dets_en = [
#     (det3x3, "First own $3 \\times 3$"),
#     (det2x2, "Second own $2 \\times 2$"),
#     (det_np_2x2, "First np $2 \\times 2$"),
#     (det_np_3x3, "Second np $3 \\times 3$")
# ]


# Determinants have numbers and are described underneath the table, as I was not able to fit it whole in LaTeX
dets = [
    (det3x3, "1."),
    (det2x2, "2."),
    (det_np_2x2, "3."),
    (det_np_3x3, "4.")
]


# Polish versions of functions
functions_pl = [
    (First_generator, 10**5, "$10^5$_z_$(-1000,_1000)$",1,True),
    (Second_generator, 10**5, "$10^5$_z_$(-10^{14},_10^{14})$",2,True),
    (Third_generator, 10**3, "Okrąg_o_promieniu_$100$_i_środku_$(0,_0)$",1,True),
    (Fourth_generator, 10**3, "Linia_$(x,_y)_=_v_\\cdot_t$,_$v_=_(0,_0.1)$,_$t_\\in_\\mathbb{R}$",0,False)
]

# English versions of functions
functions_en = [
    (First_generator,10**5,"$10^5$_from_$(-1000,_1000)$",1,True),
    (Second_generator, 10**5,"$10^5$_from_$(-10^{14},_10^{14})$",2,True),
    (Third_generator,10**3,"Circle_of_radius_$100$_and_center_$(0,_0)$",1,True),
    (Fourth_generator,10**3,"Line_$(x,_y)_=_v_\\cdot_t$,_$v_=_(0,_0.1)$,_$t_\\in_\\mathbb{R}$",0,False)
]


# Polish and English versions of points set
points_set_pl = [
    (points1,10**5,"$10^5$_z_$(-1000,_1000)$",1,True),
    (points2,10**5,"$10^5$_z_$(-10^{14},_10^{14})$",2,True),
    (points3,10**3,"Okrąg_o_promieniu_$100$_i_środku_$(0,_0)$",1,True),
    (points4,10**3,"Linia_$(x,_y)_=_v_\\cdot_t$,_$v_=_(0,_0.1)$,_$t_\\in_\\mathbb{R}$",0,False)
]


points_set_en = [
    (points1,10**5,"$10^5$_from_$(-1000,_1000)$",1,True),
    (points2, 10**5,"$10^5$_from_$(-10^{14},_10^{14})$",2,True),
    (points3,10**3,"Circle_of_radius_$100$_and_center_$(0,_0)$",1,True),
    (points4,10**3,"Line_$(x,_y)_=_v_\\cdot_t$,_$v_=_(0,_0.1)$,_$t_\\in_\\mathbb{R}$",0,False)
]

# run_all_functions(functions,epsilons,dets,types)

run_all_preprocessed(points_set_pl,epsilons,dets,types,save_to_image=WRITE_TO_FILES)